# Coordinate testing

In [1]:
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from astropy.io import fits

In [2]:
# 
df1 = pd.DataFrame(fits.getdata('../data/tim-binaries-1.fits'))
df2 = pd.DataFrame(fits.getdata('../data/tim-binaries-2.fits'))

In [10]:
import os
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
import astropy.units as u

dt = 15*u.yr # back to c. 2000
i = 59
ra = df1.ra[i]*u.deg - dt*df1.pmra[i]*u.mas/u.yr
dec = df1.dec[i]*u.deg - dt*df1.pmdec[i]*u.mas/u.yr
q = Vizier.query_region(SkyCoord(ra, dec), radius=5*u.arcsec,
                                catalog=['2mass'])
q[0]

_RAJ2000,_DEJ2000,_r,RAJ2000,DEJ2000,errMaj,errMin,errPA,_2MASS,Jmag,Jcmsig,e_Jmag,Jsnr,Hmag,Hcmsig,e_Hmag,Hsnr,Kmag,Kcmsig,e_Kmag,Ksnr,Qflg,Rflg,Bflg,Cflg,Ndet,prox,pxPA,pxCntr,Xflg,Aflg,Cntr,Hemis,Date,Scan,GLON,GLAT,Xscan,JD,Jpsfchi,Hpsfchi,Kpsfchi,Jstdap,e_Jstdap,Hstdap,e_Hstdap,Kstdap,e_Kstdap,edgeNS,edgeEW,edge,dup,use,opt,Dopt,PAopt,Bmag,Rmag,Nopt,extKey,scanKey,coaddKey,coadd,Opt
deg,deg,arcs,deg,deg,arcs,arcs,deg,,mag,mag,mag,,mag,mag,mag,,mag,mag,mag,,,,,,,arcs,deg,,,,,,,,deg,deg,arcs,d,,,,mag,mag,mag,mag,mag,mag,arcs,arcs,,,,,arcs,deg,mag,mag,,,,,,
float64,float64,float64,float64,float64,float32,float32,int16,str17,float32,float32,float32,float64,float32,float32,float32,float64,float32,float32,float32,float64,str3,str3,str3,str3,str6,float32,int16,int64,uint8,uint8,int64,str1,str10,int16,float64,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int16,str2,uint8,uint8,str1,float32,int16,float32,float32,uint8,int32,int32,int32,int16,str3
24.692811,63.717216,4.347,24.692811,63.717216,0.07,0.06,0,01384627+6343019,7.434,0.035,0.039,37053.6,6.993,0.031,0.034,22999.1,6.939,0.029,0.033,13671.6,AAA,111,111,dcc,445544,3.3,122,758578073,0,0,758578061,n,1999-09-28,114,128.143,1.326,-190.5,2451449.9246,--,--,--,--,--,--,--,--,--,8257,64,ne,1,1,T,1.2,78,10.34,9.47,4,--,41900,963686,103,Opt
24.694572,63.716732,1.154,24.694572,63.716732,0.07,0.06,0,01384669+6343002,7.440,0.021,0.027,36849.4,6.967,0.023,0.027,23556.5,6.936,0.031,0.034,13709.5,AAA,111,111,cdd,444444,3.3,302,758578061,0,0,758578073,n,1999-09-28,114,128.144,1.325,-193.3,2451449.9246,--,--,--,--,--,--,--,--,--,8259,62,ne,1,1,T,0.6,58,10.27,9.45,2,--,41900,963686,103,Opt


The closest 2mass source is still 1.15 arcsec away from where this star should be, given the current coords and the proper motion.  What's even stranger is that the Tycho-2 query doesn't seem to match up either:

In [11]:
dt = (2015-1991.5)*u.yr # back to c. 1991.5 (what the Tycho-2 website says was avg. observation epoch)
i = 59
ra = df1.ra[i]*u.deg - dt*df1.pmra[i]*u.mas/u.yr
dec = df1.dec[i]*u.deg - dt*df1.pmdec[i]*u.mas/u.yr
q = Vizier.query_region(SkyCoord(ra, dec), radius=5*u.arcsec,
                                catalog=['tycho2'])
q[0]

_RAJ2000,_DEJ2000,_r,TYC1,TYC2,TYC3,pflag,RAmdeg,DEmdeg,pmRA,pmDE,e_RAmdeg,e_DEmdeg,e_pmRA,e_pmDE,EpRAm,EpDEm,Num,q_RAmdeg,q_DEmdeg,q_pmRA,q_pmDE,BTmag,e_BTmag,VTmag,e_VTmag,prox,TYC,HIP,CCDM,RA_ICRS_,DE_ICRS_,EpRA-1990,EpDE-1990,e_RAdeg,e_DEdeg,posflg,corr
deg,deg,arcs,,,,,deg,deg,mas / yr,mas / yr,mas,mas,mas / yr,mas / yr,yr,yr,,,,,,mag,mag,mag,mag,,,,,deg,deg,yr,yr,mas,mas,,
float64,float64,float64,int16,int16,uint8,str1,float64,float64,float32,float32,int16,int16,float32,float32,float32,float32,int16,float32,float32,float32,float32,float32,float32,float32,float32,int16,str1,int32,str3,float64,float64,float32,float32,float32,float32,str1,float32
24.694258,63.716646,1.291,4035,103,2,X,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,10.501,0.032,9.533,0.021,39,T,7670,B,24.69425833,63.71664583,1.48,1.82,17.7,12.4,,-0.1


Still 1.2 arcsec away?  I must be misinterpreting the meaning of either the TGAS coordinates or the proper motions.  